In [2]:
from dotenv import load_dotenv
import os

# .env 불러오기
load_dotenv()

# 기본 모델 설정
# 여기서 설정할 수도 있지만 .env 내에 넣어줘도 됨
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [5]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Tools 설정
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    # You want to impress your coworkers with your work.
    # 위와 같은 문장을 통해 더 좋은 답변을 받을 수 있음
    # 누군가에게 감명을 준다거나 중요한 고객을 위해 일한다거나 하는 등 부연설명을 붙여주면 좋음
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,  # 다른 agent와 협업을 하지 않고 혼자 일하도록 강제하는 것
    verbose=True,
    # 사용할 Tools 명시
    tools=[
        # 구글 검색 후 링크를 가져옴
        search_tool,
        # 웹사이트에 들어가 내용을 가져옴
        scrape_tool,
    ],
    # 과도한 과금이 발생하지 않도록 조사 횟수를 제한할 수 있음
    # 제한할 수 있는 방법에는 2가지가 있음
    # 1. max_excution_time: 최대 실행 시간을 명시하여 제한
    # 2. max_iter: 답변을 출력하기 전 최대한 작업을 반복할 수 있는 횟수
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
    # editor는 delegation을 허용함으로써 컨텥트가 없다면 다른 agent에게 자료를 찾아오도록 해야 함
    allow_delegation=True,
)

task = Task(
    description="Write a blog post about {topic}",
    # Task를 수행할 agent 명시
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
)

crew = Crew(
    # 사용할 모든 agents와 tasks를 명시
    agents=[
        researcher,
        editor,
    ],
    tasks=[task],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        # flops: 기대보다 흥행에 실패한 영화
        topic="The biggest box office flops of 2024",
    ),
)

2025-02-03 18:00:15,148 - 8205137984 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Senior Writer/Editor
## Task: Write a blog post about The biggest box office flops of 2024
# Agent: Senior Researcher
## Task: Can you provide a list of the biggest box office flops of 2024, including details such as gross revenue, budget, and any notable factors that contributed to their underperformance?


# Agent: Senior Researcher
## Thought: I need to gather updated information regarding the biggest box office flops of 2024, including specific movies, their gross revenue, budgets, and factors that contributed to their underperformance.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"biggest box office flops of 2024 with details gross revenue budget reasons for failure\"}"
## Tool Output: 
{'searchParameters': {'q': 'biggest box office flops of 2024 with details gross revenue budget reasons for failure', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '10 Biggest Box Office Flops Of 2024 - Screen Rant', 'link': '